# Laboratório — Módulo I (Colab-ready)

Este notebook contém **3 exemplos executáveis** para a aula:

1) **Dados Estruturados (SQL/SQLite)** — *“Qual região possui o maior índice de assaltos nos últimos 6 meses?”*  
2) **Dados Semi-estruturados (JSON → CSV)** — consumo de API pública e transformação de campos.  
3) **Dados Não Estruturados (texto de redes sociais/conversas)** — extração de menções/hashtags e contagem simples.

> Todos os blocos são **auto-contidos** e não exigem credenciais.

In [ ]:
# (Opcional) Garantir versões recentes das libs no Colab/local
# !pip install --quiet pandas matplotlib

In [ ]:
import sqlite3, re, json, io, ssl, urllib.request
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

print("OK: libs carregadas")

## 1) Dados Estruturados — SQLite (consulta SQL)

**Pergunta socrática:** *“Qual região possui o maior índice de assaltos nos últimos 6 meses?”*

**Passos:**  
- Criar um banco **SQLite** em memória com a tabela `boletins`.  
- Inserir **dados sintéticos** (região, tipo, data).  
- Rodar a consulta SQL com `GROUP BY` e `COUNT()`.

In [ ]:
# Criar e popular base SQLite em memória
con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.execute("""
CREATE TABLE boletins (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    regiao TEXT,
    tipo TEXT,
    data_ocorrencia DATE
);
""")

# Gerar dados sintéticos (~9 meses), com "assalto" mais frequente
regioes = ["Norte", "Sul", "Leste", "Oeste", "Centro"]
tipos = ["assalto", "furto", "roubo", "ameaça"]

hoje = datetime.now().date()

import random
random.seed(42)

rows = []
for dia in range(270):  # ~9 meses
    d = hoje - timedelta(days=dia)
    for _ in range(random.randint(0, 3)):  # 0 a 3 ocorrências por dia
        reg = random.choice(regioes)
        t = random.choices(tipos, weights=[3,2,2,1])[0]  # "assalto" com maior peso
        rows.append((reg, t, d))

cur.executemany("INSERT INTO boletins (regiao, tipo, data_ocorrencia) VALUES (?, ?, ?)", rows)
con.commit()

# Consulta: últimos ~6 meses (≈180 dias)
seis_meses_atras = hoje - timedelta(days=180)

query = """
SELECT regiao, COUNT(*) AS ocorrencias
FROM boletins
WHERE tipo='assalto' AND date(data_ocorrencia) >= date(?)
GROUP BY regiao
ORDER BY ocorrencias DESC;
"""

df_sql = pd.read_sql_query(query, con, params=[seis_meses_atras.isoformat()])
df_sql

In [ ]:
# Visualizar gráfico simples (sem definir cores explicitamente)
plt.figure(figsize=(6,4))
plt.bar(df_sql['regiao'], df_sql['ocorrencias'])
plt.title("Assaltos por região — últimos ~6 meses (dados sintéticos)")
plt.xlabel("Região")
plt.ylabel("Ocorrências")
plt.tight_layout()
plt.show()

## 2) Dados Semi-estruturados — JSON → CSV

**Pergunta socrática:** *“Como automatizar a leitura e padronização de dados em JSON de diferentes fontes?”*

Neste exemplo, vamos:
- **Baixar JSON** de uma API pública (JSONPlaceholder).  
- Selecionar e **renomear campos** relevantes.  
- **Exportar para CSV** e visualizar.

In [ ]:
url = "https://jsonplaceholder.typicode.com/posts"
print("Baixando:", url)

# Contexto SSL tolerante (alguns ambientes bloqueiam certificados)
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

try:
    with urllib.request.urlopen(url, context=ctx, timeout=15) as resp:
        data = json.loads(resp.read().decode("utf-8"))
except Exception as e:
    print("Falha ao baixar; usando amostra local:", e)
    data = [
        {"userId": 1, "id": 1, "title": "Relato de ocorrência A", "body": "Texto A"},
        {"userId": 2, "id": 2, "title": "Relato de ocorrência B", "body": "Texto B"}
    ]

# Selecionar/renomear campos
registros = []
for item in data:
    registros.append({
        "usuario": item.get("userId"),
        "id": item.get("id"),
        "titulo": item.get("title"),
        "texto": item.get("body")
    })

df_json = pd.DataFrame(registros)
df_json.head()

In [ ]:
# Exportar para CSV
csv_path = "posts_padronizados.csv"
df_json.to_csv(csv_path, index=False)
print("CSV salvo em:", csv_path)

# Amostra
df_json.sample(5, random_state=42)

### Análise de Dados com `jq`

O **`jq`** é uma ferramenta de linha de comando para processar arquivos **JSON** de forma rápida e estruturada, semelhante ao uso de `grep`, `awk` ou `sed`, mas voltada para dados em formato de objetos e listas.

No contexto de **coleta e ingestão de dados**, ele é útil para:
- Inspecionar o conteúdo bruto retornado por uma **API**;
- Filtrar apenas campos relevantes (por exemplo, `id`, `title`, `userId`);
- Reformatar ou transformar a saída antes de armazenar.

A sintaxe básica é:
```bash
jq '<filtro>' arquivo.json
```

Exemplo:
```bash
jq '.[] | {usuario: .userId, titulo: .title}' posts.json
```

Dica: O jq está disponível nativamente em distribuições Linux e pode ser instalado no Google Colab com:
```bash
!apt-get install -y jq
```

A seguir, veremos um exemplo prático filtrando apenas alguns campos de um arquivo JSON exportado do nosso DataFrame.

In [ ]:
# Instalar o jq (apenas na primeira execução)
# !apt-get install -y jq > /dev/null

# Salvar o JSON da API
with open("posts.json", "w", encoding="utf-8") as f:
    json.dump(data[:5], f, ensure_ascii=False, indent=2)

# Filtrar campos usando jq
!jq -r '.[] | "\(.userId): \(.title)"' posts.json

## 3) Dados Não Estruturados — Texto (redes sociais / conversas)

**Pergunta socrática:** *“O que as redes sociais anunciam antes do boletim oficial?”*

Neste exemplo simples:
- Recebemos uma lista de **postagens** (texto livre).  
- Extraímos **menções (@)** e **hashtags (#)** com regex.  
- Contamos frequências e visualizamos.

In [ ]:
posts = [
    "Ocorrência em #Palmas ontem à noite. Patrulha já em ação. @delegacia_to",
    "Relato de furto no centro de #Araguaina. Testemunhas mencionam veículo preto. @pm_to",
    "Discussão sobre segurança em #Gurupi; evento marcado para amanhã. @org_publico",
    "Vídeo circulando de suposto assalto em #Palmas — confirmar com boletim oficial.",
    "Reforço no policiamento em #Palmas após série de furtos. @ssp_to @delegacia_to"
]

def extract_mentions_hashtags(texts):
    mentions, hashtags = [], []
    for t in texts:
        mentions += re.findall(r"@([A-Za-z0-9_\.]+)", t)
        hashtags += re.findall(r"#(\w+)", t, flags=re.UNICODE)
    return mentions, hashtags

mentions, hashtags = extract_mentions_hashtags(posts)

pd.DataFrame({
    "tipo": ["menções", "hashtags"],
    "itens": [mentions, hashtags]
})

In [ ]:
from collections import Counter

c_hashtags = Counter(hashtags)
df_hash = pd.DataFrame(c_hashtags.items(), columns=["hashtag", "freq"]).sort_values("freq", ascending=False)

c_mentions = Counter(mentions)
df_mentions = pd.DataFrame(c_mentions.items(), columns=["mencao", "freq"]).sort_values("freq", ascending=False)

print("Top hashtags:")
display(df_hash)

print("Top menções:")
display(df_mentions)

# Visualização das hashtags
plt.figure(figsize=(6,4))
plt.bar(df_hash['hashtag'], df_hash['freq'])
plt.title("Frequência de hashtags (amostra simulada)")
plt.xlabel("Hashtag")
plt.ylabel("Frequência")
plt.tight_layout()
plt.show()